In [3]:
import time

# Datasets

In [4]:
names = ['wine', 'default_credit', 'compas', 'lsat']
bnn_widths = [200, 200, 200, 200]
bnn_depths = [2, 2, 2, 2]

vae_widths = [300, 300, 300, 300] # [200, 200, 200, 200]
vae_depths = [3, 3, 3, 3] # We go deeper because we are using residual models
vae_latent_dims = [6, 8, 4, 4]

# For automatic explainer generation

regression_bools = [True, False, False, True]
gauss_cat_vae_bools = [False, True, True, True]
flat_vae_bools = [False, False, False, False]

var_names = {}
var_names_flat = {}

var_names['wine'] = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
            'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
var_names_flat['wine'] = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
            'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

var_names['default_credit'] = ['Given credit', 'Gender', 'Education', 'Marital status', 'Age', 'Payment delay 1', 'Payment delay 2',
            'Payment delay 3', 'Payment delay 4', 'Payment delay 5', 'Payment delay 6', 'Bill statement 1', 'Bill statement 2',
            'Bill statement 3', 'Bill statement 4', 'Bill statement 5', 'Bill statement 6', 'Previous payment 1', 'Previous payment 2',
            'Previous payment 3', 'Previous payment 4', 'Previous payment 5', 'Previous payment 6']
var_names_flat['default_credit'] = ['Given credit', 'Gender M', 'Gender F', 'Education grad', 'Education under', 'Education HS', 'Education Other',
                 'Marital status M', 'Marital status S', 'Marital status Other', 'Age', 'Payment delay 1', 'Payment delay 2',
            'Payment delay 3', 'Payment delay 4', 'Payment delay 5', 'Payment delay 6', 'Bill statement 1', 'Bill statement 2',
            'Bill statement 3', 'Bill statement 4', 'Bill statement 5', 'Bill statement 6', 'Previous payment 1', 'Previous payment 2',
            'Previous payment 3', 'Previous payment 4', 'Previous payment 5', 'Previous payment 6']

var_names['compas'] = ["age_cat", "race", "sex", "c_charge_degree", 'is_recid', 'priors_count', 'time_served']
var_names_flat['compas'] = ['25 - 45', 'Greater than 45', 'Less than 25', 'African-American', 'Asian', 'Caucasian', 'Hispanic', 'Native American', 'Other', 'Female', 'Male', 'Felony', 'misdemeanour', 'not_recid', 'is_recid', 'priors_count', 'time_served']

var_names['lsat'] = ['LSAT', 'UGPA', 'race', 'sex']
var_names_flat['lsat'] = ['LSAT', 'UGPA', 'amerind', 'mexican', 'other', 'black', 'asian', 'puerto', 'hisp', 'white', 'female', 'male']


# Choose dataset

In [5]:
dname = 'compas'

In [6]:
from random import seed
import numpy as np
import os
import urllib.request
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

def load_UCI(dset_name, splits=10, seed=0, separate_targets=True, save_dir='data/'):
    mkdir(save_dir)

    if dset_name == 'boston':
        if not os.path.isfile(save_dir+'housing.data'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
                               filename=save_dir+'housing.data')
        data = pd.read_csv(save_dir + 'housing.data', header=0, delimiter="\s+").values
        y_idx = [-1]

    elif dset_name == 'concrete':
        if not os.path.isfile(save_dir+'Concrete_Data.xls'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls",
                               filename=save_dir+'Concrete_Data.xls')
        data = pd.read_excel(save_dir+ 'Concrete_Data.xls', header=0, delimiter="\s+").values
        y_idx = [-1]

    elif dset_name == 'energy':
        if not os.path.isfile(save_dir+'ENB2012_data.xlsx'):
            urllib.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx",
                               filename=save_dir+'ENB2012_data.xlsx')
        data = pd.read_excel(save_dir+'ENB2012_data.xlsx', header=0, delimiter="\s+").values
        y_idx = [-2, -1]

    elif dset_name == 'power':
        if not os.path.isfile(save_dir+'CCPP.zip'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip",
                               filename=save_dir+'CCPP.zip')
        zipped = zipfile.ZipFile(save_dir+"CCPP.zip")
        data = pd.read_excel(zipped.open('CCPP/Folds5x2_pp.xlsx'), header=0, delimiter="\t").values
        y_idx = [-1]

    elif dset_name == 'wine':
        if not os.path.isfile(save_dir+'winequality-red.csv'):
            urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
                               filename=save_dir+'winequality-red.csv')
        data = pd.read_csv(save_dir+'winequality-red.csv', header=1, delimiter=';').values
        y_idx = [-1]

    elif dset_name == 'yatch':
        if not os.path.isfile(save_dir+'yacht_hydrodynamics.data'):
            urllib.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",
                               filename=save_dir+'yacht_hydrodynamics.data')
        data = pd.read_csv(save_dir+'yacht_hydrodynamics.data', header=1, delimiter='\s+').values
        y_idx = [-1]

    elif dset_name == 'kin8nm':
        if not os.path.isfile(save_dir+'dataset_2175_kin8nm.csv'):
            urllib.urlretrieve("https://www.openml.org/data/get_csv/3626/dataset_2175_kin8nm.csv",
                               filename=save_dir+'dataset_2175_kin8nm.csv')
        data = pd.read_csv(save_dir+'dataset_2175_kin8nm.csv', header=1, delimiter=',').values
        y_idx = [-1]

    elif dset_name == 'kin8nm':
        if not os.path.isfile(save_dir+'dataset_2175_kin8nm.csv'):
            urllib.urlretrieve("https://www.openml.org/data/get_csv/3626/dataset_2175_kin8nm.csv",
                               filename=save_dir+'dataset_2175_kin8nm.csv')
        data = pd.read_csv(save_dir+'dataset_2175_kin8nm.csv', header=1, delimiter=',').values
        y_idx = [-1]

    elif dset_name == 'naval':
        if not os.path.isfile(save_dir + 'UCI%20CBM%20Dataset.zip'):
            urllib.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00316/UCI%20CBM%20Dataset.zip",
                               filename=save_dir + 'UCI%20CBM%20Dataset.zip')
        zipped = zipfile.ZipFile(save_dir + "UCI%20CBM%20Dataset.zip")
        data = pd.read_csv(zipped.open('UCI CBM Dataset/data.txt'), header='infer', delimiter="\s+").values
        y_idx = [-2, -1]

    elif dset_name == 'protein':
        if not os.path.isfile(save_dir+'CASP.csv'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv",
                               filename=save_dir+'CASP.csv')
        data = pd.read_csv(save_dir+'CASP.csv', header=1, delimiter=',').values
        y_idx = [0]

    elif dset_name == 'default_credit':
        if not os.path.isfile(save_dir + 'default of credit card clients.xls'):
            urllib.request.urlretrieve(
                "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
                filename=save_dir + 'default of credit card clients.xls')
        data = pd.read_excel(save_dir + 'default of credit card clients.xls', header=[0, 1], index_col=0, # delimiter="\s+"
                             ).values
        y_idx = [-1]  # OK


    else:
        raise Exception('Dataset name doesnt match any known datasets.')

    np.random.seed(seed)
    data = data[np.random.permutation(np.arange(len(data)))]

    kf = KFold(n_splits=splits)
    for j, (train_index, test_index) in enumerate(kf.split(data)):

        if separate_targets:
            x_idx = list(range(data.shape[1]))
            for e in y_idx:
                x_idx.remove(x_idx[e])

            x_idx = np.array(x_idx)
            y_idx = np.array(y_idx)
            x_train, y_train = data[train_index, :], data[train_index, :]
            x_train, y_train = x_train[:, x_idx], y_train[:, y_idx]
            x_test, y_test = data[test_index, :], data[test_index, :]
            x_test, y_test = x_test[:, x_idx], y_test[:, y_idx]

            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)
            y_means, y_stds = y_train.mean(axis=0), y_train.std(axis=0)

            y_stds[y_stds < 1e-10] = 1
            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            y_train = ((y_train - y_means) / y_stds).astype(np.float32)

            x_test = ((x_test - x_means) / x_stds).astype(np.float32)
            y_test = ((y_test - y_means) / y_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds

        else:
            x_train, x_test = data[train_index, :], data[test_index, :]
            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)

            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            x_test = ((x_test - x_means) / x_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds


def get_my_COMPAS(rseed=0, separate_test=True, test_ratio=0.2, save_dir='../data/'):
    """
        The adult dataset can be obtained from: https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
        The code will look for the data file in the present directory, if it is not found, it will download them from GitHub.
    """

    SEED = rseed
    seed(SEED)
    np.random.seed(SEED)

    their_FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"]
    FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "c_charge_degree", "is_recid", "priors_count",
                               "time_served"]  # features to be used for classification
    CONT_VARIABLES = ["priors_count",
                      "time_served"]  # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "two_year_recid"  # the decision variable


    COMPAS_INPUT_FILE = save_dir + "compas-scores-two-years.csv"
    check_data_file(COMPAS_INPUT_FILE)

    # load the data and get some stats
    df = pd.read_csv(COMPAS_INPUT_FILE)
    df = df.dropna(subset=["days_b_screening_arrest"])  # dropping missing vals

    # convert to np array
    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])

    dates_in = data['c_jail_in']
    dates_out = data['c_jail_out']
    # this measures time in Jail
    time_served = []
    for i in range(len(dates_in)):
        di = datetime.datetime.strptime(dates_in[i], '%Y-%m-%d %H:%M:%S')
        do = datetime.datetime.strptime(dates_out[i], '%Y-%m-%d %H:%M:%S')
        time_served.append((do - di).days)
    time_served = np.array(time_served)
    time_served[time_served < 0] = 0
    data["time_served"] = time_served

    """ Filtering the data """

    # These filters are the same as propublica (refer to https://github.com/propublica/compas-analysis)
    # If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
    idx = np.logical_and(data["days_b_screening_arrest"] <= 30, data["days_b_screening_arrest"] >= -30)

    # We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
    idx = np.logical_and(idx, data["is_recid"] != -1)

    # In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
    idx = np.logical_and(idx, data["c_charge_degree"] != "O")  # F: felony, M: misconduct

    # We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
    idx = np.logical_and(idx, data["score_text"] != "NA")

    # select the examples that satisfy this criteria
    for k in data.keys():
        data[k] = data[k][idx]

    y = data[CLASS_FEATURE]
    """ Feature normalization and one hot encoding """

    print
    "\nNumber of people recidivating within two years"
    print
    pd.Series(y).value_counts()
    print
    "\n"

    X = []  # empty array with num rows same as num examples, will hstack the features to it
    X_dims = []

    feature_names = []
    for attr in FEATURES_CLASSIFICATION:
        vals = data[attr]
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            # vals = preprocessing.scale(vals, axis=0, with_mean=True, with_std=True)  # 0 mean and 1 variance
            vals = np.reshape(vals, (len(y), -1))  # convert from 1-d arr to a 2-d arr with one col
            X_dims.append(1)

        else:  # for binary categorical variables, the label binarizer uses just one var instead of two
            enc = preprocessing.OneHotEncoder(categories='auto', handle_unknown='error')
            enc.fit(vals.reshape(-1, 1))
            vals = enc.transform(vals.reshape(-1, 1)).todense()
            X_dims += [vals.shape[1]]*vals.shape[1]

        # add to learnable features
        X.append(vals)

        if attr in CONT_VARIABLES:  # continuous feature, just append the name
            feature_names.append(attr)
        else:  # categorical features
            if vals.shape[1] == 1:  # binary features that passed through lib binarizer
                feature_names.append(attr)
            else:
                for k in enc.categories_:  # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))

    X = np.array(np.concatenate(list(X), axis=1))
    X_dims = np.array(X_dims)

    if separate_test:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rseed, shuffle=True)

        x_means, x_stds = X_train.mean(axis=0), X_train.std(axis=0)
        x_means[X_dims>1] = 0
        x_stds[X_dims>1] = 1
        x_stds[x_stds < 1e-10] = 1

        x_train = ((X_train - x_means) / x_stds).astype(np.float32)
        x_test = ((X_test - x_means) / x_stds).astype(np.float32)

        return x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims
    else:
        x_means, x_stds = X.mean(axis=0), X.std(axis=0)
        print(X_dims.shape, x_means.shape)
        x_means[:,X_dims>1] = 0
        x_stds[:,X_dims>1] = 1
        x_stds[x_stds < 1e-10] = 1

        x_train = ((X - x_means) / x_stds).astype(np.float32)

        return x_train, x_means, x_stds, y, feature_names, X_dims

# Some util functions

In [7]:
def check_data_file(fname):
    files = os.listdir(".")  # get the current directory listing
    print
    "Looking for file '%s' in the current directory..." % fname

    if fname not in files:
        print
        "'%s' not found! Downloading from GitHub..." % fname
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        
        response = urllib.request.urlopen(addr)

        data = response.read()
        fileOut = open(fname, "wb")
        fileOut.write(data)
        fileOut.close()
        print
        "'%s' download and saved locally.." % fname
    else:
        print
        "File found in current directory.."

def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path, mode=0o777)
            
def unnormalise_cat_vars(x, x_means, x_stds, input_dim_vec):
    input_dim_vec = np.array(input_dim_vec)
    unnorm_x = np.multiply(x, x_stds) + x_means

    fixed_unnorm = unnorm_x.round()
    # -=
    fixed_unnorm = fixed_unnorm - fixed_unnorm.min(axis=0).reshape([1, fixed_unnorm.shape[1]])  # this sets all mins to 0
    for idx, dims in enumerate(input_dim_vec):
        if dims > 1:
            vec = fixed_unnorm[:, idx]
            vec[vec > dims - 1] = dims - 1
            fixed_unnorm[:, idx] = vec

    x[:, input_dim_vec > 1] = fixed_unnorm[:, input_dim_vec > 1]
    return x

def X_dims_to_input_dim_vec(X_dims):
    """This is for our cat_Gauss VAE model"""
    input_dim_vec = []
    i = 0
    while i < len(X_dims):
        input_dim_vec.append(X_dims[i])
        i += X_dims[i]
    return np.array(input_dim_vec)

def gauss_cat_to_flat(x, input_dim_vec):
    output = []
    for idx, dim in enumerate(input_dim_vec):
        if dim == 1:
            output.append(tf.expand_dims(x[:, idx], axis=1))
        elif dim > 1:            
            oh_vec = tf.one_hot(x[:, idx], dim)
            output.append(oh_vec)
        else:
            raise ValueError('Error, invalid dimension value')
    return tf.concat(output, axis=1).numpy()

In [8]:
if dname == 'wine':
    x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds = \
    load_UCI(dset_name='wine', splits=10, seed=42, separate_targets=True, save_dir='../data/')
    
elif dname == 'default_credit':
    # Note that this dataset is given without flattening
    x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds = \
    load_UCI(dset_name='default_credit', splits=10, seed=42, separate_targets=True, save_dir='../data/')
    print('Credit', x_train.shape, x_test.shape)
    input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1]
    
    x_train = unnormalise_cat_vars(x_train, x_means, x_stds, input_dim_vec)
    x_test = unnormalise_cat_vars(x_test, x_means, x_stds, input_dim_vec)
    # target unnormalisation
    y_train = unnormalise_cat_vars(y_train, y_means, y_stds, [2])
    y_test = unnormalise_cat_vars(y_test, y_means, y_stds, [2])
    
    x_train = gauss_cat_to_flat(torch.Tensor(x_train), input_dim_vec)
    x_test = gauss_cat_to_flat(torch.Tensor(x_test), input_dim_vec)

    print(input_dim_vec)

elif dname == 'compas':
    x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims = \
    get_my_COMPAS(rseed=42, separate_test=True, test_ratio=0.1, save_dir='../data/')
    input_dim_vec = X_dims_to_input_dim_vec(X_dims)
    print('Compas', x_train.shape, x_test.shape)
    print(input_dim_vec)
    
elif dname == 'lsat':
    x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds, my_data_keys, input_dim_vec = \
    get_my_LSAT(save_dir='../data/')
    print('LSAT', x_train.shape, x_test.shape)
    print(input_dim_vec)

Compas (5554, 17) (618, 17)
[3 6 2 2 2 1 1]


In [9]:
# import torch.utils.data as data

# class Datafeed(data.Dataset):

#     def __init__(self, x_train, y_train=None, transform=None):
#         self.data = x_train
#         self.targets = y_train
#         self.transform = transform

#     def __getitem__(self, index):
#         img = self.data[index]
#         if self.transform is not None:
#             img = self.transform(img)
#         if self.targets is not None:
#             return img, self.targets[index]
#         else:
#             return img

#     def __len__(self):
#         return len(self.data)

# # from src.utils import Datafeed

# trainset = Datafeed(x_train, y_train, transform=None)
# valset = Datafeed(x_test, y_test, transform=None)

# print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# Datasets parameters

In [10]:
names = ['wine', 'default_credit', 'compas', 'lsat']
widths = [300, 300, 300, 300] # [200, 200, 200, 200]
depths = [3, 3, 3, 3] # We go deeper because we are using residual models
latent_dims = [6, 8, 4, 4]

# BNN Loading

# VAE Loading

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_encoder(input_dim_vec,width,depth,latent_dim):
    inputs = keras.Input(shape=(sum(input_dim_vec),))
    input = layers.Dense(width)(inputs)

    for i in range(depth-1):

        x = layers.LeakyReLU()(input)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(width, kernel_initializer=tf.keras.initializers.Zeros())(x)

        x = x + input

        input = x

    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(latent_dim*2,kernel_initializer=tf.keras.initializers.Zeros())(x)

    encoder = keras.Model(inputs=inputs, outputs=outputs, name="encoder_model")

    # encoder.summary()

    # keras.utils.plot_model(encoder, "encoder.png")
    keras.utils.plot_model(encoder, "encoder.png", show_shapes=True)
    
    return encoder

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_decoder(input_dim_vec,width,depth,latent_dim):

    inputs = keras.Input(shape=(latent_dim,))
    input = layers.Dense(width,kernel_initializer=tf.keras.initializers.Zeros())(inputs)
    input = layers.LeakyReLU()(input)
    input = layers.BatchNormalization()(input)

    for i in range(depth-1):

        x = layers.LeakyReLU()(input)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(width,kernel_initializer=tf.keras.initializers.Zeros())(x)

        x = x + input

        input = x

    outputs = layers.Dense(sum(input_dim_vec),kernel_initializer=tf.keras.initializers.Zeros())(x)

    decoder = keras.Model(inputs=inputs, outputs=outputs, name="decoder_model")

    # decoder.summary()

    # keras.utils.plot_model(model, "decoder_model.png")
    keras.utils.plot_model(decoder, "decoder.png", show_shapes=True)
    
    return decoder

class VAE(tf.keras.Model):
    def __init__(self, input_dim_vec,width,depth,latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = create_encoder(input_dim_vec,width,depth,latent_dim)
        self.decoder = create_decoder(input_dim_vec,width,depth,latent_dim)

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar
    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits
    
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),axis=raxis)

def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent)#, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

@tf.function
def train_step(model, x, optimizer):

    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
def train_VAE(model):
    for epoch in range(1, epochs + 1):
        start_time = time.time()
        print(epoch,end="\n")

        ## Training
        for i,x in enumerate(x_train):
            print(f"Finished {i}/{len(x_train)}",end="\r")

            x = x.reshape(1,len(x)) # reshape(1,23)
            x_flat = gauss_cat_to_flat(x, input_dim_vec)
            train_step(model, x_flat, optimizer)

        end_time = time.time()

        ## Testing
        loss = tf.keras.metrics.Mean()
        for x in x_test:
            x = x.reshape(1,len(x)) # reshape(1,23)
            x_flat = gauss_cat_to_flat(x, input_dim_vec)
            loss(compute_loss(model, x_flat))
        elbo = -loss.result()

    #     display.clear_output(wait=False)
        print(f'Epoch: {epoch}, Test set ELBO: {elbo}, time elapse for current epoch: {end_time - start_time}')

In [12]:
epochs = 10
width = widths[names.index(dname)]
depth = depths[names.index(dname)] # number of hidden layers
latent_dim = latent_dims[names.index(dname)]
num_examples_to_generate = 16

model = VAE(input_dim_vec,width,depth,latent_dim)

2021-12-06 22:28:32.185138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 22:28:32.193536: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 22:28:32.193869: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 22:28:32.194891: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [15]:
checkpoint_path = "../VAE/checkpoints/vae"
model.load_weights(checkpoint_path)